# Multi-Dataset Bankruptcy Prediction — Full Pipeline

This notebook implements the full project specification. It includes:

1. Data ingestion (ARFF + CSVs) and annotation (`dataset_source`, `horizon_years`).
2. EDA (class ratios, missingness, distributions, correlations, PCA, per-dataset plots).
3. Safe splitting (group-aware when possible) and saving split sizes.
4. Robust preprocessing (median numeric impute, categorical string-cast, constant impute or most-frequent, dataset_source one-hot), with feature-name mapping for feature importances.
5. Baselines (Logistic Regression, Decision Tree) with per-dataset evaluation.
6. CV comparison: LightGBM (class-weight) vs LightGBM (SMOTE) with early stopping, preproc fit inside folds, SMOTE only on training fold.
7. Final model training with internal validation for early stopping, calibration (Platt), threshold tuning (accuracy subject to recall >= 0.6), and final test evaluation.
8. Feature importance mapping and SHAP analysis.
9. Saving artifacts: preproc, model, calibrated object, threshold, and a picklable `BankruptcyModel` wrapper with `predict`/`predict_proba`.
10. Reproducibility: `requirements_full.txt`, `requirements_min.txt`, and runbook.
11. Diagnostics & leakage checks, adversarial validation.


In [5]:
# 0. Setup: imports, paths, seeds
import sys, os, random, warnings
warnings.filterwarnings('ignore')
SEED = 42
random.seed(SEED)
import numpy as np; np.random.seed(SEED)
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import joblib, json, re, subprocess, traceback
from pathlib import Path

# sklearn / imblearn
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (roc_auc_score, average_precision_score, accuracy_score, f1_score,
                             precision_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve)
from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import lightgbm as lgb
import xgboost as xgb

# Paths
BASE_DIR = "bankruptcy_prediction_data"
OUT_DIR = "bankruptcy_outputs"
EDA_DIR = os.path.join(OUT_DIR, 'eda')
MODELS_DIR = os.path.join(OUT_DIR, 'models')
REPORTS_DIR = os.path.join(OUT_DIR, 'reports')
DATA_DIR = os.path.join(OUT_DIR, 'data')
for d in [OUT_DIR, EDA_DIR, MODELS_DIR, REPORTS_DIR, DATA_DIR]:
    os.makedirs(d, exist_ok=True)

print('python', sys.version)
print('pandas', pd.__version__, 'numpy', np.__version__, 'lightgbm', lgb.__version__)
print('BASE_DIR:', BASE_DIR)
print('OUT_DIR:', OUT_DIR)

python 3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:49:16) [MSC v.1929 64 bit (AMD64)]
pandas 2.2.3 numpy 2.2.6 lightgbm 4.6.0
BASE_DIR: bankruptcy_prediction_data
OUT_DIR: bankruptcy_outputs


In [6]:
# 1. Data ingestion: ARFF parser + CSVs, annotate dataset_source and horizon_years
from io import StringIO

def read_arff(path):
    txt = open(path, 'r', encoding='utf-8', errors='ignore').read()
    if '@data' not in txt.lower():
        raise ValueError(f'No @data section found in {path}')
    header, data = re.split('@data', txt, flags=re.IGNORECASE, maxsplit=1)
    attrs = []
    for line in header.splitlines():
        line = line.strip()
        if line.lower().startswith('@attribute'):
            # try different patterns
            m = re.match(r"@attribute\s+'?\"?([^'\"]+)'?\"?\s+(.*)", line, flags=re.IGNORECASE)
            if not m:
                m = re.match(r"@attribute\s+([^\s]+)\s+(.*)", line, flags=re.IGNORECASE)
            if m:
                attrs.append(m.group(1).strip())
    data_lines = [ln.strip() for ln in data.splitlines() if ln.strip() and not ln.strip().startswith('%')]
    df = pd.read_csv(StringIO('\n'.join(data_lines)), header=None, names=attrs)
    return df

# Load Poland horizons (1-5 years)
poland_files = ['1year.arff','2year.arff','3year.arff','4year.arff','5year.arff']
poland_dfs = []
for i, fname in enumerate(poland_files, start=1):
    p = os.path.join(BASE_DIR, fname)
    if not os.path.exists(p):
        print('Poland file not found:', p, ' — skipping. Ensure your BASE_DIR is correct.')
        continue
    print('Reading', p)
    df = read_arff(p)
    df['dataset_source'] = 'poland'
    df['horizon_years'] = i
    # try to detect class column
    if 'class' in df.columns:
        try:
            df['y'] = df['class'].astype(int)
        except Exception:
            # map possible labels
            df['y'] = df['class'].astype(str).str.strip().map({'1':1,'0':0,'yes':1,'no':0,'bankrupt':1}).fillna(0).astype(int)
    poland_dfs.append(df)
if poland_dfs:
    poland = pd.concat(poland_dfs, ignore_index=True, sort=False)
    print('Poland shape:', poland.shape)
else:
    poland = pd.DataFrame(); print('No Poland data loaded.')

# Taiwan
tai_path = os.path.join(BASE_DIR, 'taiwan_bankruptcy.csv')
tai = pd.DataFrame()
if os.path.exists(tai_path):
    tai = pd.read_csv(tai_path)
    tai.columns = tai.columns.str.strip().str.replace('\n',' ').str.replace('[^0-9A-Za-z_ ]','_', regex=True)
    tai['dataset_source'] = 'taiwan'; tai['horizon_years'] = 1
    bank_cols = [c for c in tai.columns if 'bankrupt' in c.lower() or 'bankrupt?' in c.lower() or 'bankruptcy' in c.lower() or 'bankrupt_' in c.lower()]
    if len(bank_cols)==0:
        # attempt common column names
        for cand in ['Bankrupt?','Bankrupt','bankrupt','Bankrupt_','bankrupt?','bankrupt_label']:
            if cand in tai.columns:
                bank_cols.append(cand)
    if len(bank_cols)==0:
        print('WARNING: No bankrupt-style column found in tai CSV; available columns:', tai.columns[:30])
    else:
        tai['y'] = tai[bank_cols[0]].astype(str).str.strip().replace({'0':'0','1':'1'}).astype(int)
    print('Taiwan shape:', tai.shape)
else:
    print('Taiwan CSV not found at', tai_path)

# US (american)
amer_path = os.path.join(BASE_DIR, 'american_bankruptcy.csv')
amer = pd.DataFrame()
if os.path.exists(amer_path):
    amer = pd.read_csv(amer_path)
    amer.columns = amer.columns.str.strip().str.replace('[^0-9A-Za-z_ ]','_', regex=True)
    amer['dataset_source'] = 'us'; amer['horizon_years'] = 1
    if 'status_label' in amer.columns:
        amer['y'] = (amer['status_label'].astype(str).str.lower()=='failed').astype(int)
    else:
        cand = [c for c in amer.columns if 'status' in c.lower() or 'label' in c.lower()]
        if cand:
            print('Using', cand[0], 'as status label')
            amer['y'] = (amer[cand[0]].astype(str).str.lower()=='failed').astype(int)
        else:
            print('WARNING: No status_label found in american CSV; columns:', amer.columns[:30])
    print('American shape:', amer.shape)
else:
    print('American CSV not found at', amer_path)

# Combine (union — keep native columns, missing columns will be NaN)
combined = pd.concat([df for df in [amer, tai, poland] if not df.empty], ignore_index=True, sort=False)
print('Combined shape:', combined.shape)
combined.to_csv(os.path.join(DATA_DIR, 'combined_raw.csv'), index=False)
print('Saved combined_raw.csv')

Reading bankruptcy_prediction_data\1year.arff
Reading bankruptcy_prediction_data\2year.arff
Reading bankruptcy_prediction_data\3year.arff
Reading bankruptcy_prediction_data\4year.arff
Reading bankruptcy_prediction_data\5year.arff
Poland shape: (43405, 68)
Taiwan shape: (6819, 99)
American shape: (78682, 24)
Combined shape: (128906, 185)
Saved combined_raw.csv


In [42]:
# 2. Extended EDA: class ratios, missingness, descriptive stats, distributions, correlations, PCA
os.makedirs(EDA_DIR, exist_ok=True)

# Class ratios per dataset
for src in combined['dataset_source'].unique():
    tmp = combined[combined['dataset_source']==src]
    rate = tmp['y'].mean() if 'y' in tmp.columns else 0
    print(f"{src} rows: {len(tmp)}, positives: {int(tmp.get('y',0).sum())}, rate: {round(rate,6)}")

# Missingness
miss = combined.isnull().mean().sort_values(ascending=False)
miss.head(50).to_csv(os.path.join(EDA_DIR, 'missingness_top50.csv'))
print('Saved missingness_top50.csv')

# Missingness heatmap (sample)
plt.figure(figsize=(12,3))
sns.heatmap(combined.sample(min(1000, len(combined)), random_state=SEED).isnull(), cbar=False)
plt.title('Missingness (sample up to 1000 rows)')
plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'missingness_heatmap.png'))
plt.close()
print('Saved missingness_heatmap.png')

# Numeric descriptive stats
num_cols = combined.select_dtypes(include=[np.number]).columns.tolist()
num_desc = combined[num_cols].describe().T
num_desc.to_csv(os.path.join(EDA_DIR, 'numeric_describe.csv'))
print('Saved numeric_describe.csv')

# Numeric distributions (up to 6 features)
numeric_cols = combined.select_dtypes(include=['number']).columns.tolist()
sample_features = numeric_cols[1:7] if len(numeric_cols)>7 else numeric_cols[:6]
for col in sample_features:
    plt.figure(figsize=(6,3))
    try:
        sns.kdeplot(data=combined, x=col, hue='y', common_norm=False, fill=True, alpha=0.3)
    except Exception:
        sns.histplot(data=combined, x=col, hue='y', element='step', stat='density', common_norm=False)
    plt.title(f'Distribution of {col} by target')
    plt.tight_layout()
    fname = os.path.join(EDA_DIR, f'dist_{col}.png')
    plt.savefig(fname); plt.close()
print('Saved sample distribution plots for features:', sample_features)

# Correlation heatmap
corr = combined.select_dtypes(include=['number']).corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Correlation heatmap (numeric)')
plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'correlation_heatmap.png')); plt.close()
print('Saved correlation_heatmap.png')

# Class-conditional means
if 'y' in combined.columns:
    class_means = combined[num_cols].groupby(combined['y']).mean().T
    class_means.to_csv(os.path.join(EDA_DIR, 'class_means.csv'))
    print('Saved class_means.csv')

# Dataset shifts for sample features
for col in sample_features:
    plt.figure(figsize=(6,3))
    sns.boxplot(data=combined, x='dataset_source', y=col)
    plt.title(f'{col} by dataset_source')
    plt.tight_layout()
    plt.savefig(os.path.join(EDA_DIR, f'box_{col}_by_dataset.png')); plt.close()
print('Saved dataset-shift boxplots')

# PCA on numeric features (sampled)
X_num = combined.select_dtypes(include='number').fillna(0)
n_sample = min(2000, X_num.shape[0])
X_sample = X_num.sample(n_sample, random_state=SEED)
y_sample = combined.loc[X_sample.index, 'y']
pca = PCA(n_components=2, random_state=SEED)
X_pca = pca.fit_transform(X_sample)
plt.figure(figsize=(6,5))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y_sample, cmap='coolwarm', alpha=0.5, s=8)
plt.title('PCA (sample) colored by bankruptcy'); plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'pca_sample.png')); plt.close()
print('Saved pca_sample.png')

us rows: 78682, positives: 5220, rate: 0.066343
taiwan rows: 6819, positives: 220, rate: 0.032263
poland rows: 43405, positives: 2091, rate: 0.048174
Saved missingness_top50.csv
Saved missingness_heatmap.png
Saved numeric_describe.csv
Saved sample distribution plots for features: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6']
Saved correlation_heatmap.png
Saved class_means.csv
Saved dataset-shift boxplots
Saved pca_sample.png


In [43]:
# 3. Data splitting strategy per spec: Stratified 80/20 per dataset (with group-aware option)
# We'll detect a likely group id (firm/company) to prevent horizon leakage in Poland.
possible_group_cols = [c for c in combined.columns if c.lower() in ('firm_id','company','id','ticker','name','company_id','firm')]
group_col = possible_group_cols[0] if possible_group_cols else None
print('Detected group column (if any):', group_col)

train_parts = []
test_parts = []
for src in combined['dataset_source'].unique():
    df_src = combined[combined['dataset_source']==src].copy().reset_index(drop=True)
    if group_col and group_col in df_src.columns:
        # group-aware split: ensure groups are not shared across train/test
        gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
        try:
            tr_idx, te_idx = next(gss.split(df_src, groups=df_src[group_col]))
        except Exception as e:
            print('Group split failed for', src, e); tr_idx, te_idx = train_test_split(df_src.index, test_size=0.2, stratify=df_src['y'] if 'y' in df_src else None, random_state=SEED)
    else:
        if 'y' in df_src.columns and df_src['y'].nunique()>1 and df_src.shape[0]>=10:
            tr_idx, te_idx = train_test_split(df_src.index, test_size=0.2, stratify=df_src['y'], random_state=SEED)
        else:
            tr_idx, te_idx = train_test_split(df_src.index, test_size=0.2, random_state=SEED)
    train_parts.append(df_src.loc[tr_idx])
    test_parts.append(df_src.loc[te_idx])

train_df = pd.concat(train_parts, ignore_index=True, sort=False)
test_df  = pd.concat(test_parts,  ignore_index=True, sort=False)
print('Train_df shape:', train_df.shape, 'Test_df shape:', test_df.shape)

# Save split sizes
sizes = {}
for src in combined['dataset_source'].unique():
    sizes[src] = {'total': int((combined['dataset_source']==src).sum()),
                  'train': int((train_df['dataset_source']==src).sum()),
                  'test': int((test_df['dataset_source']==src).sum()),
                  'positives_total': int(((combined['dataset_source']==src) & (combined['y']==1)).sum()) if 'y' in combined.columns else None}
with open(os.path.join(REPORTS_DIR, 'split_sizes.json'), 'w') as f:
    json.dump(sizes, f, indent=2)
print('Saved split_sizes.json')

Detected group column (if any): None
Train_df shape: (103124, 185) Test_df shape: (25782, 185)
Saved split_sizes.json


In [44]:
# Collinearity pruning before preprocessing
# Keep only numeric columns
numeric_cols = train_df.select_dtypes(include=['number']).columns.tolist()

# Compute correlation matrix
corr_matrix = train_df[numeric_cols].corr().abs()

# Upper triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop highly correlated features (threshold = 0.95)
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
print("Dropping highly correlated numeric features:", to_drop)

# Optionally drop from train/test
train_df = train_df.drop(columns=to_drop)
test_df  = test_df.drop(columns=to_drop, errors='ignore')

Dropping highly correlated numeric features: ['X9', 'X12', 'X16', 'X17', 'X18', 'Bankrupt_', 'ROA_B_ before interest and depreciation after tax', 'Realized Sales Gross Margin', 'After_tax net Interest Rate', 'Continuous interest rate _after tax_', 'Net Value Per Share _A_', 'Net Value Per Share _C_', 'Per Share Net profit before tax _Yuan __', 'Regular Net Profit Growth Rate', 'Net worth_Assets', 'Operating profit_Paid_in capital', 'Net profit before tax_Paid_in capital', 'Working capitcal Turnover Rate', 'Cash Flow to Sales', 'Current Liability to Liability', 'Current Liability to Equity', 'Net Income to Total Assets', 'Gross Profit to Sales', 'Liability to Equity', 'class']


In [45]:
# 4. Preprocessing: preserve native features; one-hot dataset_source; median impute numerics within pipeline
from sklearn.preprocessing import FunctionTransformer

# Identify numeric and categorical columns in train_df (excluding target)
drop_cols = ['y']
all_cols = [c for c in train_df.columns if c not in drop_cols]

numeric_cols = train_df.select_dtypes(include=['number']).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != 'y']

cat_cols = [c for c in all_cols if c not in numeric_cols and c!='dataset_source']

# We'll one-hot encode dataset_source explicitly
ohe_cols = ['dataset_source']

# helper
def to_str_func(X):
    return X.astype(str)

# numeric transformer (median)
num_tf = Pipeline([('imputer', SimpleImputer(strategy='median'))])

# safe OneHotEncoder fallback: use sparse_output if available else sparse
ohe_params = {}
try:
    OneHotEncoder(sparse_output=False)
    ohe_params['sparse_output'] = False
except Exception:
    ohe_params['sparse'] = False

ohe_tf = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore', **ohe_params))])

# categorical transformer: cast to str, impute constant 'missing', then ordinal
cat_tf = Pipeline([
    ('to_str', FunctionTransformer(to_str_func)),
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preproc = ColumnTransformer([
    ('num', num_tf, numeric_cols),
    ('ohe_src', ohe_tf, ohe_cols),
    ('cat', cat_tf, cat_cols)
], remainder='drop')

print('Numeric features:', len(numeric_cols), 'Other cat features:', len(cat_cols))
# For linear baselines, a separate pipeline with OneHot for categories may be constructed later.

Numeric features: 92 Other cat features: 66


In [46]:
# 5. Baselines: Logistic Regression (with scaling) and Decision Tree
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

X_tr = train_df.drop(columns=['y']); y_tr = train_df['y']
X_te = test_df.drop(columns=['y']);  y_te = test_df['y']

# Ensure all categorical columns are strings
for c in cat_cols + ohe_cols:
    X_tr[c] = X_tr[c].astype(str)
    X_te[c] = X_te[c].astype(str)

# Split categorical columns into high-cardinality and low-cardinality
high_card_cols = [c for c in cat_cols + ohe_cols if X_tr[c].nunique() > 20]
low_card_cols = [c for c in cat_cols + ohe_cols if X_tr[c].nunique() <= 20]

# Logistic Regression pipeline: numeric impute + scale, OHE for low-card, TargetEncode high-card
preproc_lr = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler(with_mean=False))  # with_mean=False for sparse compatibility
    ]), numeric_cols),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=True), low_card_cols),
    ('target', TargetEncoder(), high_card_cols)
], remainder='drop')

pipe_lr = Pipeline([
    ('preproc', preproc_lr),
    ('clf', LogisticRegression(
        max_iter=1000, class_weight='balanced', solver='saga', random_state=SEED
    ))
])

# Decision Tree pipeline: same preproc as main model
pipe_dt = Pipeline([
    ('preproc', preproc),
    ('clf', DecisionTreeClassifier(max_depth=6, class_weight='balanced', random_state=SEED))
])

# Fit baselines
print('Training Logistic Regression...')
pipe_lr.fit(X_tr, y_tr)
print('Training Decision Tree...')
pipe_dt.fit(X_tr, y_tr)

# Evaluate and save function
def eval_and_save(pipe, name):
    # Combined evaluation
    proba = pipe.predict_proba(X_te)[:,1]
    preds = pipe.predict(X_te)
    res = {
        'accuracy': float(accuracy_score(y_te, preds)),
        'roc_auc': float(roc_auc_score(y_te, proba)),
        'pr_auc': float(average_precision_score(y_te, proba)),
        'f1': float(f1_score(y_te, preds)),
        'precision': float(precision_score(y_te, preds, zero_division=0)),
        'recall': float(recall_score(y_te, preds)),
        'confusion': confusion_matrix(y_te, preds).tolist()
    }
    with open(os.path.join(REPORTS_DIR, f'{name}_report.json'), 'w') as f:
        json.dump(res, f, indent=2)

    # Per-dataset evaluation
    per_dataset = {}
    for src in test_df['dataset_source'].unique():
        tmp = test_df[test_df['dataset_source']==src]
        if tmp.shape[0]==0: continue
        Xtmp = tmp.drop(columns=['y'])
        ytmp = tmp['y']
        p = pipe.predict_proba(Xtmp)[:,1]
        preds_tmp = pipe.predict(Xtmp)
        per_dataset[src] = {
            'accuracy': float(accuracy_score(ytmp, preds_tmp)),
            'roc_auc': float(roc_auc_score(ytmp, p)),
            'pr_auc': float(average_precision_score(ytmp, p)),
            'f1': float(f1_score(ytmp, preds_tmp)),
            'precision': float(precision_score(ytmp, preds_tmp, zero_division=0)),
            'recall': float(recall_score(ytmp, preds_tmp)),
            'confusion': confusion_matrix(ytmp, preds_tmp).tolist()
        }
    with open(os.path.join(REPORTS_DIR, f'{name}_per_dataset.json'), 'w') as f:
        json.dump(per_dataset, f, indent=2)

    # Save pipeline
    joblib.dump(pipe, os.path.join(MODELS_DIR, f'pipe_{name}.joblib'))
    print(f'Saved {name} report and model.')

# Run evaluation and save
eval_and_save(pipe_lr, 'logistic_regression')
eval_and_save(pipe_dt, 'decision_tree')

Training Logistic Regression...
Training Decision Tree...
Saved logistic_regression report and model.
Saved decision_tree report and model.


In [47]:
# 6. Compare LightGBM with class-weights vs LightGBM with SMOTE inside CV (and optional XGBoost)
from sklearn.model_selection import StratifiedKFold
import math, time

def cv_compare_models(X, y, preproc, use_smote_flag=False, model_type='lgb', n_splits=5, early_stopping_rounds=50):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    results = []
    fold = 1
    for train_idx, val_idx in skf.split(X, y):
        try:
            X_train_raw, X_val_raw = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            # Fit preproc on training fold only
            preproc.fit(X_train_raw)
            X_train = preproc.transform(X_train_raw)
            X_val   = preproc.transform(X_val_raw)

            # apply SMOTE only after preprocessing
            if use_smote_flag:
                sm = SMOTE(random_state=SEED)
                X_train, y_train = sm.fit_resample(X_train, y_train)

            if model_type == 'lgb':
                pos = int(np.sum(y_train)); neg = len(y_train)-pos
                clf = lgb.LGBMClassifier(objective='binary', random_state=SEED,
                                         n_estimators=2000, learning_rate=0.03, n_jobs=-1)
                if not use_smote_flag:
                    clf.set_params(scale_pos_weight=float(neg)/max(1.0,pos))
                clf.fit(
                        X_train, y_train,
                        eval_set=[(X_val, y_val)],
                        eval_metric='auc',
                        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False)]
                      )
            else:
                pos = int(np.sum(y_train)); neg = len(y_train)-pos
                clf = xgb.XGBClassifier(objective='binary:logistic', random_state=SEED,
                                        n_estimators=2000, learning_rate=0.03, tree_method='hist', n_jobs=-1)
                if not use_smote_flag:
                    clf.set_params(scale_pos_weight=float(neg)/max(1.0,pos))
                clf.fit(
                        X_train, y_train,
                        eval_set=[(X_val, y_val)],
                        eval_metric='auc',
                        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False)]
                       )

            proba = clf.predict_proba(X_val)[:,1]
            preds = (proba >= 0.5).astype(int)

            fold_metrics = {
                'accuracy': accuracy_score(y_val, preds),
                'recall': recall_score(y_val, preds),
                'precision': precision_score(y_val, preds, zero_division=0),
                'f1': f1_score(y_val, preds),
                'roc_auc': roc_auc_score(y_val, proba),
                'pr_auc': average_precision_score(y_val, proba)
            }
            results.append(fold_metrics)
            print(f' Fold {fold} ({"SMOTE" if use_smote_flag else "ClassWeight"}) AUC={fold_metrics["roc_auc"]:.4f} Acc={fold_metrics["accuracy"]:.3f}')
        except Exception as e:
            print(' Fold', fold, 'failed:', e)
            traceback.print_exc()
        fold += 1

    if results:
        mean_metrics = {k: float(np.mean([r[k] for r in results])) for k in results[0]}
        std_metrics  = {k: float(np.std([r[k] for r in results]))  for k in results[0]}
    else:
        mean_metrics = std_metrics = {m: None for m in ['accuracy','recall','precision','f1','roc_auc','pr_auc']}
    return {'mean': mean_metrics, 'std': std_metrics, 'n_valid_folds': len(results)}

print('Running CV: LGBM class-weight...')
r_lgb_class = cv_compare_models(X_tr, y_tr, preproc, use_smote_flag=False, model_type='lgb', n_splits=5)
print('Running CV: LGBM SMOTE...')
r_lgb_smote  = cv_compare_models(X_tr, y_tr, preproc, use_smote_flag=True,  model_type='lgb', n_splits=5)

# Optional XGB comparison (comment/uncomment to run)
# print('Running CV: XGB class-weight...')
# r_xgb_class = cv_compare_models(X_tr, y_tr, preproc, use_smote_flag=False, model_type='xgb', n_splits=5)
# print('Running CV: XGB SMOTE...')
# r_xgb_smote  = cv_compare_models(X_tr, y_tr, preproc, use_smote_flag=True,  model_type='xgb', n_splits=5)

cv_comp = {'lgb_class_weight': r_lgb_class, 'lgb_smote': r_lgb_smote}
with open(os.path.join(REPORTS_DIR, 'cv_smote_vs_classweight.json'), 'w') as f:
    json.dump(cv_comp, f, indent=2)
print('Saved CV comparison to cv_smote_vs_classweight.json')

Running CV: LGBM class-weight...
[LightGBM] [Info] Number of positive: 4820, number of negative: 77679
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 39038
[LightGBM] [Info] Number of data points in the train set: 82499, number of used features: 159
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058425 -> initscore=-2.779811
[LightGBM] [Info] Start training from score -2.779811
 Fold 1 (ClassWeight) AUC=0.9724 Acc=0.967
[LightGBM] [Info] Number of positive: 4820, number of negative: 77679
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047749 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 39038
[LightGBM] [Info] Number of data p

In [48]:
# 7. Final model training: choose based on CV (ROC-AUC). Default to class-weight unless SMOTE is better.
from lightgbm import early_stopping, log_evaluation

with open(os.path.join(REPORTS_DIR, 'cv_smote_vs_classweight.json')) as f:
    cv_comp = json.load(f)

lgb_class_auc = cv_comp['lgb_class_weight']['mean']['roc_auc']
lgb_smote_auc  = cv_comp['lgb_smote']['mean']['roc_auc']
use_smote = bool(lgb_smote_auc and lgb_smote_auc > lgb_class_auc)
print('CV mean ROC-AUC - LGB(class):', lgb_class_auc, ' LGB(smote):', lgb_smote_auc, ' -> use_smote=', use_smote)

# Fit preproc on full training set and transform
preproc.fit(X_tr)
X_tr_t = preproc.transform(X_tr)
X_te_t = preproc.transform(X_te)

# internal split for early stopping and calibration
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(X_tr_t, y_tr, test_size=0.15, stratify=y_tr, random_state=SEED)

if use_smote:
    sm = SMOTE(random_state=SEED)
    X_train_res, y_train_res = sm.fit_resample(X_train_final, y_train_final)
else:
    X_train_res, y_train_res = X_train_final, y_train_final

final_clf = lgb.LGBMClassifier(objective='binary', random_state=SEED,
                               n_estimators=5000, learning_rate=0.03, n_jobs=-1)
if not use_smote:
    pos = int(np.sum(y_train_res)); neg = len(y_train_res)-pos
    final_clf.set_params(scale_pos_weight=float(neg)/max(1.0,pos))

final_clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val_final, y_val_final)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=100, verbose=True),
        log_evaluation(period=100)  # prints evaluation every 100 rounds
    ]
)

# Save uncalibrated pipeline (preproc is fitted)
final_pipeline = make_pipeline(preproc, final_clf)
joblib.dump(final_pipeline, os.path.join(MODELS_DIR, 'final_pipeline_uncalibrated.joblib'))
print('Saved final_pipeline_uncalibrated.joblib')

CV mean ROC-AUC - LGB(class): 0.9703803982667664  LGB(smote): 0.9783930978655933  -> use_smote= True
[LightGBM] [Info] Number of positive: 82534, number of negative: 82534
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.142654 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 39660
[LightGBM] [Info] Number of data points in the train set: 165068, number of used features: 160
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.993544	valid_0's binary_logloss: 0.0865195
[200]	valid_0's auc: 0.995532	valid_0's binary_logloss: 0.0470927
[300]	valid_0's auc: 0.995895	valid_0's binary_logloss: 0.0393419
[400]	valid_0's auc: 0.996004	valid_0's binary_logloss: 0.0354012
[500]	valid_0's auc: 0.996122	valid_0's binary_logloss: 0.0329929
[600]	valid_0's auc: 0.996254	valid_0's binary_logloss: 0.031235


In [49]:
# Step 8: Calibration (Platt/sigmoid), threshold tuning, and calibration plot
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

calibrator = CalibratedClassifierCV(estimator=final_clf, cv='prefit', method='sigmoid')
calibrator.fit(X_val_final, y_val_final)

# Threshold tuning: maximize accuracy while ensuring recall in [0.6, 0.7]
probs_val = calibrator.predict_proba(X_val_final)[:,1]
best_acc, best_thr = -1, 0.5

for thr in np.linspace(0.01, 0.99, 99):
    preds = (probs_val >= thr).astype(int)
    rec = recall_score(y_val_final, preds)
    acc = accuracy_score(y_val_final, preds)
    if 0.6 <= rec <= 0.7 and acc > best_acc:
        best_acc, best_thr = acc, thr

# fallback if no threshold satisfies constraint: pick closest recall >0.6
if best_acc < 0:
    recalls = [recall_score(y_val_final, (probs_val >= thr).astype(int)) for thr in np.linspace(0.01,0.99,99)]
    # choose threshold with recall just above 0.6 and highest accuracy
    candidate_idxs = [i for i,r in enumerate(recalls) if r >= 0.6]
    if candidate_idxs:
        candidate_accs = [accuracy_score(y_val_final, (probs_val >= np.linspace(0.01,0.99,99)[i]).astype(int)) for i in candidate_idxs]
        best_idx = candidate_idxs[np.argmax(candidate_accs)]
        best_thr = np.linspace(0.01,0.99,99)[best_idx]
        best_acc = accuracy_score(y_val_final, (probs_val >= best_thr).astype(int))
    else:
        # fallback: use 0.5
        best_thr = 0.5
        best_acc = accuracy_score(y_val_final, (probs_val >= best_thr).astype(int))

print(f"Chosen threshold (rec in 0.6-0.7): {best_thr:.3f}, Validation accuracy: {best_acc:.3f}")

# Save calibration plot
prob_true, prob_pred = calibration_curve(y_val_final, probs_val, n_bins=10)
plt.figure(figsize=(5,4))
plt.plot(prob_pred, prob_true, marker='o', label='Calibrated')
plt.plot([0,1],[0,1],'--', color='gray', label='Perfect')
plt.title('Calibration curve (validation)')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'calibration_curve_validation.png'))
plt.close()
print("Saved calibration_curve_validation.png")

# Save artifact: preproc, final classifier, calibrator, and threshold
artifact = {
    'preproc': preproc,
    'final_clf': final_clf,
    'calibrator': calibrator,
    'threshold': float(best_thr),
    'use_smote': use_smote,
    'cv_comp': cv_comp
}
joblib.dump(artifact, os.path.join(MODELS_DIR, 'final_artifact_calibrated.joblib'))
print("Saved final_artifact_calibrated.joblib")

Chosen threshold (rec in 0.6-0.7): 0.680, Validation accuracy: 0.992
Saved calibration_curve_validation.png
Saved final_artifact_calibrated.joblib


In [50]:
# 9. Final evaluation: combined + per-dataset; save reports and plots
# Predict on test set
probs_test = artifact['calibrator'].predict_proba(X_te_t)[:,1]
preds_test = (probs_test >= artifact['threshold']).astype(int)

final_report = {
    'accuracy': float(accuracy_score(y_te, preds_test)),
    'roc_auc': float(roc_auc_score(y_te, probs_test)),
    'pr_auc': float(average_precision_score(y_te, probs_test)),
    'f1': float(f1_score(y_te, preds_test)),
    'precision': float(precision_score(y_te, preds_test, zero_division=0)),
    'recall': float(recall_score(y_te, preds_test)),
    'confusion': confusion_matrix(y_te, preds_test).tolist()
}
with open(os.path.join(REPORTS_DIR, 'final_test_evaluation.json'), 'w') as f:
    json.dump(final_report, f, indent=2)
print('Saved final_test_evaluation.json:', final_report)

cm = confusion_matrix(y_te, preds_test)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'confusion_matrix_test.png'))
plt.close()
print("Saved confusion_matrix_test.png")

# per-dataset evaluation
per_dataset = {}
for src in test_df['dataset_source'].unique():
    tmp = test_df[test_df['dataset_source']==src]
    if tmp.shape[0]==0: continue
    Xtmp = tmp.drop(columns=['y']); ytmp = tmp['y']
    xt = preproc.transform(Xtmp)
    p = artifact['calibrator'].predict_proba(xt)[:,1]
    preds_tmp = (p >= artifact['threshold']).astype(int)
    per_dataset[src] = {
        'accuracy': float(accuracy_score(ytmp, preds_tmp)),
        'roc_auc': float(roc_auc_score(ytmp, p)),
        'pr_auc': float(average_precision_score(ytmp, p)),
        'f1': float(f1_score(ytmp, preds_tmp)),
        'precision': float(precision_score(ytmp, preds_tmp, zero_division=0)),
        'recall': float(recall_score(ytmp, preds_tmp)),
        'confusion': confusion_matrix(ytmp, preds_tmp).tolist()
    }
with open(os.path.join(REPORTS_DIR, 'per_dataset_evaluation.json'), 'w') as f:
    json.dump(per_dataset, f, indent=2)
print('Saved per_dataset_evaluation.json')

for src in test_df['dataset_source'].unique():
    tmp = test_df[test_df['dataset_source']==src]
    if tmp.shape[0]==0: continue
    Xtmp = tmp.drop(columns=['y']); ytmp = tmp['y']
    xt = preproc.transform(Xtmp)
    p = artifact['calibrator'].predict_proba(xt)[:,1]
    preds_tmp = (p >= artifact['threshold']).astype(int)
    
    cm = confusion_matrix(ytmp, preds_tmp)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {src}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig(os.path.join(EDA_DIR, f'confusion_matrix_{src}.png'))
    plt.close()
print("Saved per-dataset confusion matrix plots")

# ROC and PR curves (combined)
fpr, tpr, _ = roc_curve(y_te, probs_test)
plt.figure(figsize=(5,4)); plt.plot(fpr, tpr); plt.title('ROC Curve (test)'); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.tight_layout(); plt.savefig(os.path.join(EDA_DIR, 'roc_curve_test.png')); plt.close()
prec, rec, _ = precision_recall_curve(y_te, probs_test)
plt.figure(figsize=(5,4)); plt.plot(rec, prec); plt.title('PR Curve (test)'); plt.xlabel('Recall'); plt.ylabel('Precision'); plt.tight_layout(); plt.savefig(os.path.join(EDA_DIR, 'pr_curve_test.png')); plt.close()
print('Saved ROC and PR curves')

# Per-dataset ROC/PR plots
for src in test_df['dataset_source'].unique():
    tmp = test_df[test_df['dataset_source']==src]
    if tmp.shape[0]==0: continue
    Xtmp = tmp.drop(columns=['y']); ytmp = tmp['y']
    xt = preproc.transform(Xtmp)
    p = artifact['calibrator'].predict_proba(xt)[:,1]
    fpr, tpr, _ = roc_curve(ytmp, p)
    plt.figure(figsize=(5,4)); plt.plot(fpr, tpr); plt.title(f'ROC (test) - {src}'); plt.tight_layout(); plt.savefig(os.path.join(EDA_DIR, f'roc_{src}.png')); plt.close()
    prec, rec, _ = precision_recall_curve(ytmp, p)
    plt.figure(figsize=(5,4)); plt.plot(rec, prec); plt.title(f'PR (test) - {src}'); plt.tight_layout(); plt.savefig(os.path.join(EDA_DIR, f'pr_{src}.png')); plt.close()
print('Saved per-dataset ROC/PR plots')

Saved final_test_evaluation.json: {'accuracy': 0.9824296020479404, 'roc_auc': 0.9770307466787981, 'pr_auc': 0.8810972070769384, 'f1': 0.825836216839677, 'precision': 0.9808219178082191, 'recall': 0.7131474103585658, 'confusion': [[24255, 21], [432, 1074]]}
Saved confusion_matrix_test.png
Saved per_dataset_evaluation.json
Saved per-dataset confusion matrix plots
Saved ROC and PR curves
Saved per-dataset ROC/PR plots


In [51]:
# 10. Feature importance
def get_feature_names(column_transformer):
    # adapted from various recipes to get feature names after ColumnTransformer
    output_features = []
    for name, trans, cols in column_transformer.transformers_:
        if name == 'remainder' and trans == 'drop':
            continue
        if hasattr(trans, 'named_steps'):
            # pipeline inside
            last = trans.named_steps[list(trans.named_steps.keys())[-1]]
            if hasattr(last, 'get_feature_names_out'):
                try:
                    names = last.get_feature_names_out(cols)
                except Exception:
                    try:
                        names = last.get_feature_names_out()
                    except Exception:
                        names = cols
                output_features.extend([f"{name}__{n}" for n in names])
            else:
                # fallback: use original column names (for ordinal etc.)
                if isinstance(cols, (list, tuple, pd.Index)):
                    output_features.extend([f"{name}__{c}" for c in cols])
                else:
                    output_features.append(f"{name}__{cols}")
        else:
            # transformer is a single estimator
            if hasattr(trans, 'get_feature_names_out'):
                try:
                    names = trans.get_feature_names_out(cols)
                except Exception:
                    names = cols
                output_features.extend([f"{name}__{n}" for n in names])
            else:
                if isinstance(cols, (list, tuple, pd.Index)):
                    output_features.extend([f"{name}__{c}" for c in cols])
                else:
                    output_features.append(f"{name}__{cols}")
    return output_features

try:
    feat_names = get_feature_names(preproc)
    print('Number of transformed features:', len(feat_names))
except Exception as e:
    print('Could not extract transformed feature names:', e); feat_names = None

# LGBM feature importances
try:
    model = final_clf
    if hasattr(model, 'booster_'):
        booster = model.booster_
        imp_vals = booster.feature_importance(importance_type='gain')
        names = booster.feature_name()
    elif hasattr(model, 'feature_importances_'):
        imp_vals = model.feature_importances_
        names = feat_names if feat_names is not None else [f'f{i}' for i in range(len(imp_vals))]
    else:
        imp_vals = None; names = None

    if imp_vals is not None and names is not None:
        feat_imp = sorted(list(zip(names, imp_vals)), key=lambda x: x[1], reverse=True)[:50]
        with open(os.path.join(REPORTS_DIR, 'feature_importance_top50.json'), 'w') as f:
            json.dump([{k:v} for k,v in feat_imp], f, indent=2)
        print('Saved feature_importance_top50.json')
except Exception as e:
    print('Feature importance extraction failed:', e)

Number of transformed features: 161
Saved feature_importance_top50.json


In [52]:
# 11. Build a picklable sklearn-like wrapper for deployment: BankruptcyModel
class BankruptcyModel:
    def __init__(self, preproc, calibrator, clf, threshold, meta=None):
        self.preproc = preproc
        self.calibrator = calibrator
        self.clf = clf
        self.threshold = float(threshold)
        self.meta = meta or {}
    def predict_proba(self, X_raw):
        Xt = self.preproc.transform(X_raw)
        return self.calibrator.predict_proba(Xt)
    def predict(self, X_raw):
        probs = self.predict_proba(X_raw)[:,1]
        return (probs >= self.threshold).astype(int)
    def save(self, path):
        joblib.dump(self, path)
    @classmethod
    def load(cls, path):
        return joblib.load(path)

# create and save final model wrapper
bm = BankruptcyModel(preproc=artifact['preproc'], calibrator=artifact['calibrator'], clf=artifact['final_clf'], threshold=artifact['threshold'], meta={'use_smote':artifact['use_smote'], 'cv':artifact['cv_comp']})
bm.save(os.path.join(MODELS_DIR, 'bankruptcy_model.pkl'))
print('Saved picklable BankruptcyModel as bankruptcy_model.pkl')

Saved picklable BankruptcyModel as bankruptcy_model.pkl


In [53]:
# 12. Reproducibility: requirements and runbook
# requirements_full via pip freeze
req_min = [
    f"python=={sys.version.split()[0]}",
    f"pandas=={pd.__version__}",
    f"numpy=={np.__version__}",
    f"scikit-learn=={__import__('sklearn').__version__}",
    f"lightgbm=={lgb.__version__}",
    f"xgboost=={xgb.__version__}",
    f"imbalanced-learn=={__import__('imblearn').__version__}",
    "matplotlib", "seaborn", "joblib"
]
with open(os.path.join(OUT_DIR, 'requirements.txt'), 'w') as f:
    f.write('\n'.join(req_min))
print('Saved requirements.txt')

# runbook
runbook = f"""Runbook:
1. Ensure data files are in BASE_DIR: {BASE_DIR}
2. Run cells sequentially.
3. Outputs are saved under: {OUT_DIR}
4. Models: {MODELS_DIR}/bankruptcy_model.pkl and final artifacts.
5. CV results: {REPORTS_DIR}/cv_smote_vs_classweight.json
6. For troubleshooting, inspect {EDA_DIR} and {REPORTS_DIR}
"""
with open(os.path.join(OUT_DIR, 'runbook.txt'), 'w') as f:
    f.write(runbook)
print('Saved runbook.txt')

Saved requirements.txt
Saved runbook.txt


In [54]:
# 13. Diagnostics: checks for leakage or suspiciously-perfect metrics
# 1) features perfectly determining y (including categories)
for c in train_df.columns:
    if c == 'y': continue
    try:
        vals = train_df[[c,'y']].dropna()
        if vals.shape[0] == 0: continue
        mapping = vals.groupby(c)['y'].nunique()
        if any(mapping == 1) and len(mapping) <= 100:
            print('WARNING: Column', c, 'may fully determine y for some values; sample counts:')
            print(vals.groupby(c)['y'].value_counts().unstack(fill_value=0).head())
    except Exception:
        pass

# 2) Duplicate firm ids across splits (if group_col exists)
if 'group_col' in globals() and group_col:
    gtr = set(train_df[group_col].dropna().unique())
    gte = set(test_df[group_col].dropna().unique())
    overlap = gtr & gte
    print('Group overlap train/test (should be 0):', len(overlap))
    if len(overlap)>0:
        print('Example overlapping groups:', list(overlap)[:10])

# 3) adversarial validation (train vs test)
try:
    adv = pd.concat([train_df.assign(is_holdout=0), test_df.assign(is_holdout=1)], ignore_index=True)
    adv_X = adv.drop(columns=['y','is_holdout'])
    adv_y = adv['is_holdout']
    adv_preproc = preproc
    adv_preproc.fit(adv_X)
    adv_Xt = adv_preproc.transform(adv_X)
    adv_clf = LogisticRegression(max_iter=1000, random_state=SEED)
    adv_clf.fit(adv_Xt, adv_y)
    adv_pred = adv_clf.predict_proba(adv_Xt)[:,1]
    print('Adversarial validation ROC-AUC (train vs test):', roc_auc_score(adv_y, adv_pred))
except Exception as e:
    print('Adversarial validation failed:', e)

y                 0     1
status_label             
alive         58769     0
failed            0  4176
Adversarial validation ROC-AUC (train vs test): 0.4999916697475963
